# Installing Necessary Libraries

## pip install ipywidgets ipython requests beautifulsoup4 pandas matplotlib seaborn numpy

## csv, time, random, and math are standard Python libraries, so you don't need to install them via pip. You can safely omit them from the installation command.

# Importing Libraries

In [ ]:
import ipywidgets as widgets             # ipywidgets provides interactive widgets for Jupyter notebooks
from IPython.display import display       # display from IPython is used to show widgets and other elements in Jupyter notebooks
import requests                           # requests library allows sending HTTP requests to fetch web data
from bs4 import BeautifulSoup             # BeautifulSoup is used to parse HTML content for web scraping
import csv                                # csv library is used for reading and writing CSV files
import time                               # time library for adding delays between requests to avoid getting blocked
import random                             # random library for generating random numbers, useful for setting random delays
import pandas as pd                       # pandas is a powerful library for data manipulation and analysis
import matplotlib.pyplot as plt           # matplotlib is used for creating static, animated, and interactive visualizations
import seaborn as sns                     # seaborn is a data visualization library based on matplotlib with a focus on statistical plots
import math                               # math library provides mathematical functions and constants
import numpy as np                        # numpy is a library for handling arrays and numerical data operations

# Define the category options

In [ ]:
# Define the category options for a dropdown or selection widget
categories = [
    "--Select Category--",                         # Default option for users to prompt a category selection
    "Electronics - Smartphones",                   # Option for selecting smartphones under Electronics
    "Cosmetics - Moisturizing Face Cream",         # Option for selecting moisturizing face cream under Cosmetics
    "Health & Beauty - Organic Shampoos for Dry Hair",  # Option for selecting organic shampoos for dry hair under Health & Beauty
    "Computers & Accessories - Laptops",           # Option for selecting laptops under Computers & Accessories
    "Home & Kitchen - Non-Stick Pans"              # Option for selecting non-stick pans under Home & Kitchen
]


# Defining The URL (Uniform Resource Locator)

In [ ]:
# Defining the URLs corresponding to each category in the categories list
urls = [
    "#",  # Placeholder URL, can be replaced with a valid URL if needed
    "https://www.amazon.in/s?k=smartphones&ref=nb_sb_noss",  # URL for Electronics - Smartphones
    "https://www.amazon.in/s?k=moisturizing+face+cream&ref=nb_sb_noss",  # URL for Cosmetics - Moisturizing Face Cream
    "https://www.amazon.in/s?k=organic+shampoo+for+dry+hair&ref=nb_sb_noss",  # URL for Health & Beauty - Organic Shampoos for Dry Hair
    "https://www.amazon.in/s?k=laptops&ref=nb_sb_noss",  # URL for Computers & Accessories - Laptops
    "https://www.amazon.in/s?k=non-stick+pans&ref=nb_sb_noss"  # URL for Home & Kitchen - Non-Stick Pans
]


# Create a dropdown for category selection

In [ ]:
# Create a dropdown widget for selecting a category from the available options
category_dropdown = widgets.Dropdown(
    options=categories,               # Populate the dropdown with the categories list
    description="Category:",          # Label for the dropdown widget
    disabled=False,                   # Allow the dropdown to be interactive (not disabled)
)

# Shared state to keep track of the selected URL based on the chosen category
state = {'selected_url': None}

# Function to handle category selection changes and update the state
def on_category_change(change):
    chosen_category = change['new']    # Get the new category selected by the user
    choice_index = categories.index(chosen_category)   # Find the index of the selected category

    # Update the shared state with the corresponding URL based on the selected category
    state['selected_url'] = urls[choice_index]

    # Print the selected category and its corresponding URL
    print(f"You selected: {chosen_category}")
    print(f"Your chosen category URL is: {state['selected_url']}")

# Attach the function to the category dropdown so it triggers when the value changes
category_dropdown.observe(on_category_change, names='value')

# Display the dropdown widget to the user
display(category_dropdown)


# Start the scraping process after URL selection

In [ ]:
# Exponential backoff with jitter (randomization) to avoid overwhelming the server
def wait_retry(retry_count):
    base_wait_time = 2 ** retry_count  # Calculate the base wait time using exponential backoff
    random_wait_time = random.uniform(base_wait_time, base_wait_time * 2)  # Add random variation to avoid synchronized retries
    print(f"Retrying in {random_wait_time:.2f} seconds...")  # Inform the user of the retry wait time
    time.sleep(random_wait_time)  # Wait for the calculated time before retrying

# Function to scrape Amazon based on the selected URL
def scrape_amazon(selected_url, max_retries=5):  # Allow up to 5 retries for the request
    retry_count = 0  # Initialize retry counter

    # Loop to retry the request if it fails, up to max_retries
    while retry_count < max_retries:
        try:
            # Define HTTP headers to mimic a real browser request
            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
                "Accept-Language": "en-US,en;q=0.5",
                "Accept-Encoding": "gzip, deflate, br",
                "Connection": "keep-alive",
                "Referer": "https://www.amazon.in/"
            }
            response = requests.get(selected_url, headers=headers)  # Send GET request to the selected URL
            response.raise_for_status()  # Raise an exception if the response status indicates an error (4xx or 5xx)

            # Parse the HTML content of the page using BeautifulSoup
            soup = BeautifulSoup(response.content, "html.parser")

            # Function to extract and clean product title by removing extra text in parentheses
            def extract_product_title(description):
                return description.split('(')[0].strip()

            # Function to ensure product title is between 10-20 characters; truncate if too long
            def clean_title(title):
                if 10 <= len(title) <= 20:
                    return title.strip()
                if len(title) > 20:
                    return title[:20].strip()  # Limit title to 20 characters if it's too long
                else:
                    return title  # Keep original title if it's shorter than 10 characters

            # Open (or create) a CSV file to store the scraped data
            with open('amazon_products.csv', mode='w', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)

                # Write the header row for the CSV file with column names
                writer.writerow(['S.NO', 'PRODUCT NAME', 'PRODUCT ID', 'PRICE', 'RATING', 'NUMBER OF REVIEWS', 'DELIVERY DATE', 'PRODUCT DESCRIPTION'])

                # Find all product listings on the page
                products = soup.find_all('div', {'data-component-type': 's-search-result'})

                # Initialize a serial number counter for each product entry
                serial_number = 1

                # Loop through each product element and extract relevant details
                for product in products:
                    # Extract product ID (ASIN) if available, otherwise use "-"
                    product_id = product.get('data-asin', '-')

                    # Extract raw product title, clean it, and limit the length if needed
                    raw_title = product.h2.text.strip() if product.h2 else "-"
                    extracted_name = extract_product_title(raw_title)
                    name = clean_title(extracted_name)

                    # Extract price if available, clean it, remove commas, or use "0" if unavailable
                    price = product.find('span', 'a-price-whole')
                    price = price.text.strip() if price else "-"
                    price = price.replace(",", "") if price != "-" else 0  # Remove commas and convert to float

                    # Extract rating if available, otherwise default to "0"
                    rating = product.find('span', 'a-icon-alt')
                    rating = rating.text.split(" ")[0].strip() if rating else "0"

                    # Extract number of reviews if available, otherwise default to "0"
                    reviews = product.find('span', {'class': 'a-size-base'})
                    reviews = reviews.text.strip() if reviews and reviews.text.strip().isdigit() else "0"

                    # Extract delivery information if available, otherwise use a default message
                    delivery = product.find('span', {'class': 'a-color-base a-text-bold'})
                    delivery = delivery.text.strip() if delivery else "Delivery info not available"

                    # Write the extracted product data as a new row in the CSV file
                    writer.writerow([serial_number, name, product_id, float(price), float(rating), int(reviews), delivery, raw_title])

                    # Increment the serial number for the next product entry
                    serial_number += 1

            # Load the CSV file data into a DataFrame for further processing and analysis
            df = pd.read_csv('amazon_products.csv')

            # Calculate median values for price, rating, and reviews, ignoring zero values
            median_price = df.loc[df['PRICE'] != 0, 'PRICE'].median()
            median_rating = df.loc[df['RATING'] != 0, 'RATING'].median()
            median_reviews = np.ceil(df.loc[df['NUMBER OF REVIEWS'] != 0, 'NUMBER OF REVIEWS'].median())  # Round up to nearest integer

            # Replace zero values in the DataFrame with the calculated median values
            df['PRICE'] = df['PRICE'].replace(0, median_price)  # Ensure PRICE always has 2 decimal places
            df['RATING'] = df['RATING'].replace(0, median_rating)  # Ensure RATING always has 1 decimal place
            df['NUMBER OF REVIEWS'] = df['NUMBER OF REVIEWS'].replace(0, median_reviews).astype(int)  # Convert to integer

            # Save the updated DataFrame back to the CSV file
            df.to_csv('amazon_products.csv', index=False)

            print("Scraping complete. Data saved to amazon_products.csv")
            break  # Exit the retry loop if scraping is successful

        except requests.exceptions.HTTPError as http_err:
            # Handle specific HTTP errors like 503, which indicates server unavailability
            if response.status_code == 503:
                print(f"HTTP 503 error occurred: {http_err}. Retrying...")
            else:
                print(f"HTTP error occurred: {http_err}")
        except requests.exceptions.ConnectionError as conn_err:
            # Handle connection errors
            print(f"Connection error occurred: {conn_err}")
        except requests.exceptions.Timeout as timeout_err:
            # Handle timeout errors
            print(f"Timeout error occurred: {timeout_err}")
        except requests.exceptions.RequestException as req_err:
            # Handle other general request exceptions
            print(f"An error occurred: {req_err}")

        retry_count += 1
        wait_retry(retry_count)  # Wait and retry with exponential backoff after each failure

    else:
        print("Max retries reached, scraping failed.")  # Report failure after reaching max retries

# Trigger the scraping process only if a URL has been selected
if 'selected_url' in state and state['selected_url']:
    scrape_amazon(state['selected_url'])
else:
    print("No category selected. Please select a category to start scraping.")


# Displaying Scrapped Data in a Table Format Using Pandas

In [ ]:
# Read the CSV file containing the scraped Amazon product data into a Pandas DataFrame
df = pd.read_csv('amazon_products.csv')

# Format 'PRICE' and 'RATING' columns to have specific decimal places
# Ensure 'PRICE' always has two decimal places, 'RATING' has one decimal place,
# and 'NUMBER OF REVIEWS' is rounded to the nearest integer

# Apply lambda function to format 'PRICE' to two decimal pl}aces as a string
df['PRICE'] = df['PRICE'].apply(lambda x: f"{x:.2f}")

# Apply lambda function to format 'RATING' to one decimal place as a string
df['RATING'] = df['RATING'].apply(lambda x: f"{x:.1f}")

# Round 'NUMBER OF REVIEWS' to the nearest integer and convert to integer type
df['NUMBER OF REVIEWS'] = df['NUMBER OF REVIEWS'].round().astype(int)

# Define a function to apply alternating background colors to DataFrame rows
def highlight_alternating_rows(x):
    # List to store style rules for each row in the DataFrame
    styles = []

    # Loop through each row in the DataFrame to apply alternating row colors
    for i in range(len(x)):
        if i % 2 == 0:  # Check if row index is even
            # Apply a light green background and black text for even rows
            styles.append('background-color: #e6f7e6; color: #000000')
        else:  # Row index is odd
            # Apply a slightly darker green background and black text for odd rows
            styles.append('background-color: #ccffcc; color: #000000')
    return styles  # Return the list of styles for the entire column

# Apply custom styling to the DataFrame and store it in 'styled_df'

styled_df = (
    df.head(100000)  # Limit the DataFrame to the first 100,000 rows for styling
    .style.apply(highlight_alternating_rows, axis=0)  # Apply alternating colors row-wise
    .set_properties(**{'font-size': '12pt', 'font-family': 'Arial'})  # Set font size and family for all cells
    .set_table_styles([
        # Style table headers (th): set bold font, large font size, font family, and add a border
        {'selector': 'th', 'props': [('font-size', '16pt'), ('font-weight', 'bold'), ('font-family', 'Cambria'), ('border', '1px solid black')]},

        # Style data cells (td): add borders around each cell and center-align text
        {'selector': 'td', 'props': [('border', '1px solid black'), ('text-align', 'center')]},

        # Add an outer border around the table for visual separation
        {'selector': 'table', 'props': [('border', '2px solid black')]}
    ])
)

# Display the styled DataFrame within a Jupyter notebook or interactive environment
display(styled_df)


# Visualization Starts

# Necessary Conditions

In [ ]:
# Load the data from the CSV file named 'amazon_products.csv' into a DataFrame called 'data'
data = pd.read_csv("amazon_products.csv")

# Calculate the total number of products scraped by finding the length of the DataFrame
num_products = len(data)

# Adjust the figure size dynamically based on the number of products scraped
# The width is set to 'num_products * 0.9' to scale proportionally with the number of products
# The height is set to the maximum of either 6 or 'num_products / 3.0', ensuring a minimum height of 6
fig_size = (num_products * 0.9, max(6, num_products / 3.0))

# Print a completion message indicating the number of products scraped
print(f"After completion of the scraping process, {num_products} products were scraped.")

# Print the adjusted figure size, which will be used for visualizations or plots
print(f"Figure size adjusted to: {fig_size}")


# Representing Visualization "Product Vs Price Distribution Of Each Product"

In [ ]:
# Load the data from the CSV file named 'amazon_products.csv' into a DataFrame
data = pd.read_csv("amazon_products.csv")

# Setting up the figure size for the plot based on the fig_size variable
plt.figure(figsize=fig_size)

# Using the "viridis" color palette from Seaborn to style the bars with a gradient color scheme
sns.set_palette("viridis")

# Generate a sequence of numerical indices for each product to use as x-axis labels
indices = np.arange(len(data))

# Create a bar chart where:
# - x-axis represents product indices
# - y-axis represents product prices from the 'PRICE' column
# - each bar is colored using the 'viridis' color palette
bars = plt.bar(indices, data['PRICE'], color=sns.color_palette("viridis", len(data)))

# Adding labels above each bar to display the price of each product
for i, bar in enumerate(bars):
    # Calculate the x and y coordinates for the label:
    # - x: place the label at the center of the bar
    # - y: position the label just above the height of the bar
    x = bar.get_x() + bar.get_width() / 2
    y = bar.get_height()

    # Format the price with a rupee symbol, commas, and two decimal places
    price_label = f'₹{bar.get_height():,.2f}'

    # Add the price label text above the bar with:
    # - horizontal alignment at the center (ha='center')
    # - vertical alignment at the bottom (va='bottom')
    # - specified font size (font_size) for readability
    plt.text(x, y, price_label,
             ha='center', va='bottom', fontsize=10,
             rotation=0)  # rotation=0 ensures horizontal text

# Set x-axis label to 'PRODUCT ID' with font size 15 for clarity
plt.xlabel('PRODUCT ID', fontsize=15)

# Set y-axis label to 'PRICE' with font size 15 for clarity
plt.ylabel('PRICE', fontsize=15)

# Set the title of the plot with a larger font size for emphasis
plt.title("Price Distribution of Each Product", fontsize=20)

# Map the 'PRODUCT ID' column to the x-axis tick labels, positioning them at each index
# Rotate labels by 45 degrees and align them to the right to prevent overlap
plt.xticks(indices, data['PRODUCT ID'], rotation=45, ha='right', fontsize=10)

# Disable grid lines on the plot for a cleaner look
plt.grid(False)

# Adjust the layout to ensure there is enough spacing to prevent label overlap
plt.tight_layout()

# Display the plot in the notebook or environment
plt.show()

# Save the figure as a PNG file named 'price_distribution.png'
# - dpi=300 for high-quality resolution
# - bbox_inches='tight' to fit all elements within the saved image
plt.savefig('price_distribution.png', dpi=300, bbox_inches='tight')


# Representing Visualization "Product Vs Number Of Reviews"

In [ ]:
# Reading the CSV file containing Amazon product data into a DataFrame
data = pd.read_csv("amazon_products.csv")

# Converting the 'NUMBER OF REVIEWS' column to numeric values, and setting invalid values to NaN
data['NUMBER OF REVIEWS'] = pd.to_numeric(data['NUMBER OF REVIEWS'], errors='coerce')

# Handling missing or NaN values in the 'PRODUCT NAME' column:
# - Filling NaN values with "Unknown Product" for better readability
# - Converting all product names to strings and removing leading/trailing spaces
data['PRODUCT NAME'] = data['PRODUCT NAME'].fillna("Unknown Product").astype(str).str.strip()

# Sorting the DataFrame by the 'NUMBER OF REVIEWS' column in descending order for better visualization
data = data.sort_values(by='NUMBER OF REVIEWS', ascending=False)

# Limiting the number of products to plot if needed
limit = len(data)  # Plot all products
data_subset = data.head(limit)  # Select the top `limit` products for plotting

# Setting up numerical indices for the x-axis to correspond with product names
x_indices = np.arange(len(data_subset))

# Setting up the figure size for the plot based on the fig_size variable
plt.figure(figsize=fig_size)

# Plotting the line plot with blue markers ('bo-') for each product's number of reviews:
# - `x_indices` provides x-axis positions for each product
# - `data_subset['NUMBER OF REVIEWS']` represents the y-values (number of reviews)
plt.plot(x_indices, data_subset['NUMBER OF REVIEWS'], 'bo-', linewidth=1)

# Adding a filled area under the line plot for visual effect:
# - Color is set to light blue with partial transparency (alpha=0.5)
plt.fill_between(x_indices, data_subset['NUMBER OF REVIEWS'], color='lightblue', alpha=0.5)

# Setting the plot's title, x-axis label, and y-axis label with specified font sizes
plt.title('Products vs Number of Reviews (Line Plot)', fontsize=20)
plt.xlabel('PRODUCT NAME', fontsize=15)
plt.ylabel('NUMBER OF REVIEWS', fontsize=15)

# Mapping product names to the x-axis ticks at each numerical index:
# - Rotating labels by 45 degrees for readability
# - Aligning labels to the right
plt.xticks(x_indices, data_subset['PRODUCT NAME'], rotation=45, ha='right', fontsize=10)

# Adding data labels to each point on the line plot to show the number of reviews
for i, review_count in enumerate(data_subset['NUMBER OF REVIEWS']):
    # Adding the annotation at each point:
    # - `xy` is the point location
    # - `xytext` specifies the offset above each point for the label
    plt.annotate(f'{review_count}', (i, review_count), textcoords="offset points", xytext=(0, 5), ha='center')

# Adjusting layout to prevent label and title overlap
plt.tight_layout()

# Enabling grid lines for better readability of data points
plt.grid(True)

# Displaying the line plot
plt.show()

# Saving the figure as a PNG file named 'reviews_distribution.png'
# - dpi=300 for high resolution
# - bbox_inches='tight' ensures all plot elements fit within the image boundaries
plt.savefig('reviews_distribution.png', dpi=300, bbox_inches='tight')

# Representing Visualization "Rating Distribution Of Each Product"

In [ ]:
# Load the data from the CSV file
data = pd.read_csv('amazon_products.csv')

# Convert the 'RATING' column to string type before applying string operations.
# This helps handle any non-numeric values, like "N/A," to avoid errors during extraction.
data['RATING'] = data['RATING'].astype(str)

# Extract only the rating number (before "out of") and convert it to a numeric type.
# The regular expression `r'(\d+\.\d+)'` captures decimal ratings (e.g., extracts "4.5" from "4.5 out of 5").
data['RATING'] = data['RATING'].str.extract(r'(\d+\.\d+)').astype(float)

# Count the occurrences of each rating value.
# `value_counts()` tallies each unique rating, and `sort_index()` arranges them in ascending order.
rating_counts = data['RATING'].value_counts().sort_index()

# Set the Seaborn color palette to 'pastel' for a visually appealing, contrasting pie chart.
sns.set_palette("pastel")

# Create the pie chart figure with the specified figure size (`fig_size`).
plt.figure(figsize=fig_size)

# Generate a color palette based on the number of unique ratings.
# This ensures each rating segment has a unique pastel color.
colors = sns.color_palette("pastel", n_colors=len(rating_counts))

# Plot the pie chart with improved labels, percentage display, and segment borders.
plt.pie(rating_counts,
        labels=[f'Rating: {x} ({y} products)' for x, y in zip(rating_counts.index, rating_counts)],  # Label format
        autopct='%1.1f%%',  # Display percentage with one decimal point
        startangle=90,  # Start the pie chart rotation from the top
        colors=colors,  # Use Seaborn's pastel color palette for segment colors
        textprops={'fontsize': 12, 'color': 'black'},  # Text properties: font size and color
        wedgeprops={'edgecolor': 'black', 'linewidth': 0.25})  # Borders around each pie segment

# Add a title to the pie chart with larger font size for readability.
plt.title('Rating Distribution Across Products', fontsize=20)

# Ensure the pie chart is drawn as a circle.
plt.axis('equal')

# Display the pie chart on the screen.
plt.show()

# Save the figure as a PNG file named 'rating_distribution.png' for high-resolution output.
# `dpi=300` for quality, and `bbox_inches='tight'` to ensure nothing is cropped.
plt.savefig('rating_distribution.png', dpi=300, bbox_inches='tight')

# Representing Visualization "Top 20 Most Expensive Products Based on Price"

In [ ]:
# Load the data from the CSV file
df = pd.read_csv("amazon_products.csv")

# Get the top 20 products sorted by price in descending order
# `sort_values(by=['PRICE'], ascending=False)` sorts the data in descending order by the 'PRICE' column,
# and `head(20)` selects the top 20 rows.
top_products_by_price = df.sort_values(by=['PRICE'], ascending=False).head(20)

# Plotting a bar chart for the top 20 products based on price, in descending order.
plt.figure(figsize=fig_size)  # Set the figure size for the chart

# Create a barplot with prices on the x-axis and product names on the y-axis
bars = sns.barplot(
    x=top_products_by_price['PRICE'],  # X-axis shows the prices of the products
    y=top_products_by_price['PRODUCT NAME'],  # Y-axis shows the product names
    palette='viridis',  # Use the 'viridis' color palette for visual appeal
    hue=top_products_by_price['PRICE'],  # Color gradient based on product prices
    dodge=False,  # Set dodge to False since we need only one bar per product
    legend=False  # Remove the legend since color gradient is used only for visualization
)

# Track the bars that already have a label
labeled_bars = set()

# Add labels to each bar in the bar chart to display product prices
for bar in bars.patches:
    # Get the width (price) and height of each bar
    width = bar.get_width()
    height = bar.get_height()

    # Check if the bar has been labeled already
    if bar.get_y() not in labeled_bars:
        # Calculate position for the label to display price on the right side of the bar
        offset_x = 0.025 * width  # Adjust this multiplier to control spacing from the bar
        x = width + offset_x  # Slightly offset the label from the bar
        y = bar.get_y() + height / 2  # Center the label vertically within the bar

        # Format the price label with commas for thousands and two decimal places
        price_label = f'₹{width:,.2f}'

        # Add the text label to the chart with adjusted horizontal alignment and vertical alignment
        plt.text(x, y, price_label, ha='left', va='center', fontsize=10)

        # Mark this bar as labeled
        labeled_bars.add(bar.get_y())

# Label and format the chart for clarity
plt.xlabel("Price (in ₹)", fontsize=15)  # Label the x-axis
plt.ylabel("Product Name", fontsize=15)  # Label the y-axis
plt.title("Top Most Expensive Products Based on Price", fontsize=20)  # Title for the bar chart
plt.yticks(rotation=45, ha='right')  # Rotate y-axis labels for readability

# Display the plot on screen
plt.show()

# Save the figure as a PNG file named 'top_most_expensive_products.png' with high resolution
# `dpi=300` for quality and `bbox_inches='tight'` to ensure nothing is cropped.
plt.savefig('top_most_expensive_products.png', dpi=300, bbox_inches='tight')


# Representing Visualization "Bottom 20 Least Expensive Products Based on Price"

In [ ]:
# Load the data from the CSV file
df = pd.read_csv("amazon_products.csv")

# Get the bottom 20 products sorted by price in ascending order
bottom_products_by_price = df.sort_values(by=['PRICE'], ascending=True).head(20)

# Plotting a bar chart for bottom 20 products based on price, sorted in ascending order
plt.figure(figsize=fig_size)  # Set the figure size
bars = sns.barplot(
    x=bottom_products_by_price['PRICE'],  # X-axis: Price of the products
    y=bottom_products_by_price['PRODUCT NAME'],  # Y-axis: Product names
    palette='viridis',  # Color palette for the bars
    hue=bottom_products_by_price['PRICE'],  # Use PRICE for the color gradient
    dodge=False,  # Do not dodge the bars (only one bar per product)
    legend=False  # Remove the legend since hue is used only for color gradient
)

# Iterating through bars and adding labels
for bar in bars.patches:
    # Get bar width and height
    width = bar.get_width()
    height = bar.get_height()

    # Calculate x and y position for the label
    x = width + 0.1  # Add a small offset to position label outside the bar
    y = bar.get_y() + height / 2  # Vertically center the label

    # Format the price value with commas and 2 decimal places
    price_label = f'₹{width:,.2f}'

    # Add the text label
    plt.text(x, y, price_label, ha='left', va='center', fontsize=10)

# Labeling and formatting the chart
plt.xlabel("Price (in ₹)",fontsize=15)  # X-axis label for price
plt.ylabel("Product Name",fontsize=15)  # Y-axis label for product names
plt.title("Bottom Least Expensive Products Based on Price",fontsize=20)  # Title of the chart
plt.yticks(rotation=45, ha='right')  # Rotate the y-axis labels for better readability
plt.show()  # Display the plot

# Save the figure in PNG file Format
plt.savefig('bottom_least_expensive_products.png', dpi=300, bbox_inches='tight')
